# 智能借貸

追蹤智能借貸 https://docs.compound.finance/#networks 一個月內的以下數據

**題目 A：鉅額清算分析 (The "Absorb" Event)**
Compound V3 的清算機制叫做 "Absorption" (吸收)，協議會把壞帳買下來。

- **目標**：找出過去一個月內最大的幾筆清算事件。

**題目 B：巨鯨行為追蹤 (User Position Health)**

- **目標**：監控某巨鯨在 Compound V3 上的健康係數。

## introduce

the ETH compoundV3 contract url `https://etherscan.io/address/0xc3d688B66703497DAA19211EEdff47f25384cdc3`

### methods

ref: `https://docs.compound.finance/collateral-and-borrowing/`

1. withdraw: 提款，帳戶餘額不足即為借貸
2. supply: 存入款項
3. zap: 基於 supply 的打包，會把其他資產用流動池轉成 USDC 後 supply
4. AbsorbCollateral: 用基礎貨幣購買抵押物
5. AbsorbDebt: 平帳，用抵押物償還基礎貨幣
6. getBorrowRate: 當前利率
7. collateralBalanceOf: 抵押物餘額

analysis:

ref: `https://github.com/compound-finance/comet/blob/main/contracts/Comet.sol`

with ETH V3 protocol (0xc3d688B66703497DAA19211EEdff47f25384cdc3)

V3 的清償 process:

1. 出借方用 Supply (0xd1cf3d156d5f8f0d50f6c122ed609cec09d35c9b9fb3fff6ea0959134dae424e) 把 USDC 存入 contract
    - (inner) 會把幣打包成 compoundV3 USDC 給出借方作證明
2. 借款方用 SupplyCollateral (0xfa56f7b24f17183d81894d3ac2ee654e3c26388d17a28dbd9549b8114304e1f4) 把 其它幣 存入 contract
    - (inner) 會把抵押物打包成 Wrapped XXXX 給借款方作證明
3. 借款方用 withdraw (0x9b1bfa7fa9ee420a16e124f794c35ac9f90472acc99140eb2f6447c714cad8eb) 把 USDC 借出
4. 審計者用 absorb 觸發以下兩個方法，把抵押物收納進合約本身，借款償還
    - AbsorbCollateral (0x9850ab1af75177e4a9201c65a2cf7976d5d28e40ef63494b44366f86b2f9412e)
    - AbsorbDebt (0x1547a878dc89ad3c367b6338b4be6a65a5dd74fb77ae044da1e8747ef1f4f62f)

others:

- WithdrawCollateral (0xd6d480d5b3068db003533b170d67561494d72e3bf9fa40a266471351ebba9e16)，提出抵押物
- Transfer (0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef)，輔助方法
- Approve (0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925)，允許抵押物轉入
- BuyCollateral (0xf891b2a411b0e66a5f0a6ff1368670fefa287a13f541eb633a386a1a9cc7046b)，外部購買合約持有的抵押物

to analysis, we dump ETH contract `0xc3d688B66703497DAA19211EEdff47f25384cdc3` all event in 1 month

by SQL

```sql
SELECT
  transaction_hash,
  topics[SAFE_OFFSET(0)] as method,
  topics[SAFE_OFFSET(1)] as offset1,
  topics[SAFE_OFFSET(2)] as offset2,
  topics[SAFE_OFFSET(3)] as offset3,
  data as value,
  block_timestamp
FROM `bigquery-public-data.crypto_ethereum.logs`
WHERE
  address = LOWER('0xc3d688B66703497DAA19211EEdff47f25384cdc3')
  AND (DATE(block_timestamp) >= '2026-1-18' AND DATE(block_timestamp) < '2026-2-20')
```

## Question A

想找到大型違約，方法就是掃描 absorbDebt 事件

接著分析事件原始碼: `https://github.com/compound-finance/comet/blob/ed6ebcd84ac00906e8e725716891d482f4bef8b9/contracts/CometMainInterface.sol#L46`

```sol
event AbsorbDebt(address indexed absorber, address indexed borrower, uint basePaidOut, uint usdValue);
```

其中格式如下

- 含 index 的可以放在 topic
    - topic0: AbsorbDebt = 0x1547a878dc89ad3c367b6338b4be6a65a5dd74fb77ae044da1e8747ef1f4f62f
    - topic1: absorber
    - topic2: borrower
- 不含 index 放 value，uint = 32B = 64 char
  - basePaidOut (基礎貨幣數)
  - usdValue (當時美金價值)


In [5]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

df = pd.read_json("bquxjob_63548f2f_19c805378f2.json", lines=False)
df = df.sort_values(
    by=['block_timestamp'],
    ascending=[True]
)

df = df.query("method == '0x1547a878dc89ad3c367b6338b4be6a65a5dd74fb77ae044da1e8747ef1f4f62f'").copy()
print(len(df))
df['usdValue_str'] = df['value'].apply(lambda x: x[len(x)-64:len(x)])
df['usdValue_number'] = df['usdValue_str'].apply(lambda x: int(x, 16))
df = df.sort_values(
    by=['usdValue_number'],
    ascending=[False]
)
# 把債務最多的 tx 放入 eth scan 搜索即可查看詳情
print(df.head())

152
                                                       transaction_hash  \
130  0x477d838a5d58f4adb8eaa590df600753190cc805ad191be05047c3e863f14ac2   
40   0x16c813b0eb19f8604fb86c8826afcb1861f145e3f3c62dc3aeabf1f6cf2a7f3e   
137  0x2e1a0aee9557f917d393095d0d614ee1c428b4aef77edac45734eaa790db7a3d   
35   0x603f4cf77545d02f857e97b8598ab16228442ca8166380dce468c527da9b2e5d   
61   0xde8da5c098cc13ac4ae198d2dc43d82dc61d476ae0e99768739f87310059a2a7   

                                                                 method  \
130  0x1547a878dc89ad3c367b6338b4be6a65a5dd74fb77ae044da1e8747ef1f4f62f   
40   0x1547a878dc89ad3c367b6338b4be6a65a5dd74fb77ae044da1e8747ef1f4f62f   
137  0x1547a878dc89ad3c367b6338b4be6a65a5dd74fb77ae044da1e8747ef1f4f62f   
35   0x1547a878dc89ad3c367b6338b4be6a65a5dd74fb77ae044da1e8747ef1f4f62f   
61   0x1547a878dc89ad3c367b6338b4be6a65a5dd74fb77ae044da1e8747ef1f4f62f   

                                                                offset1  \
130  0x000000000000

# Question B

直接嘗試追蹤上一筆查詢出來，被大額清償的帳戶: `https://etherscan.io/address/0xb7b7eb7e9611975bc9715f22ce7e6ee288296fd4`

從 eth scan 上看，確實是個非常活躍的 ETH account

此處一樣要從原始碼中得知事件定義: https://github.com/compound-finance/comet/blob/ed6ebcd84ac00906e8e725716891d482f4bef8b9/contracts/CometMainInterface.sol#L41

要完整計算健康係數需要得知用戶的完整持有資產，但這在短時間內的 log 尚無法追蹤，因此這裡的做法單純是試著取出資產與抵押物之關西

- SupplyCollateral / WithdrawCollateral: 追蹤 $CollateralBalance_i$。
- Supply / Withdraw (Base Asset): 追蹤 $TotalBorrowValue$。注意 Compound V3 使用


In [7]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

df = pd.read_json("bquxjob_63548f2f_19c805378f2.json", lines=False)
df = df.sort_values(
    by=['block_timestamp'],
    ascending=[True]
)
whale_address = '0x000000000000000000000000b7b7eb7e9611975bc9715f22ce7e6ee288296fd4'

# 使用 @ 引用變數
df = df.query("offset1 == @whale_address or offset2 == @whale_address").copy()

df_supply = df.query("method == '0xd1cf3d156d5f8f0d50f6c122ed609cec09d35c9b9fb3fff6ea0959134dae424e'").copy()
df_with_draw = df.query("method == '0x9b1bfa7fa9ee420a16e124f794c35ac9f90472acc99140eb2f6447c714cad8eb'").copy()

df_supply_collateral = df.query("method == '0xfa56f7b24f17183d81894d3ac2ee654e3c26388d17a28dbd9549b8114304e1f4'").copy()
df_with_draw_collateral = df.query("method == '0xd6d480d5b3068db003533b170d67561494d72e3bf9fa40a266471351ebba9e16'").copy()

total_supply = df_supply['value'].apply(lambda x: int(x, 16)).sum() - df_with_draw['value'].apply(lambda x: int(x, 16)).sum()
total_collateral = {}
for row in df_supply_collateral.itertuples():
    # print(row)
    if total_collateral.get(row.offset3):
        total_collateral[row.offset3] += int(row.value, 16)
    else:
        total_collateral[row.offset3] = int(row.value, 16)

for row in df_with_draw_collateral.itertuples():
    # print(row)
    if total_collateral.get(row.offset3):
        total_collateral[row.offset3] -= int(row.value, 16)
    else:
        total_collateral[row.offset3] = -int(row.value, 16)

print(total_collateral) # {[抵押物地址]: 抵押物數量}
print(total_supply)     # 存入資產 (基於基礎幣值)

# 結果如下，這個月內存入許多 ETH，並且借出不少 USDC，但看來因為 ETH 價格崩盤而被清算
# {'0x000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2': 150000000000000000}
# -673147878303


{'0x000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2': 150000000000000000}
-673147878303
